In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Input,Dropout,GaussianNoise
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import StratifiedKFold,RepeatedStratifiedKFold,train_test_split,KFold
from sklearn.decomposition import PCA
from pandas import DataFrame
from time import time
import numpy as np
import h5py
from docopt import docopt
from utils import  load_fold

In [2]:
x_train, y_train,x_valid, y_valid,x_test, y_test=[],[],[],[],[],[]
X,Y=[],[]
experiments=['cc200_whole']
with h5py.File("data/sample.hdf5".encode('utf-8'), "r") as hdf5:
        
        for experiment in experiments:
            exp_storage=hdf5["experiments/cc200_whole"]
            for fold in exp_storage:
                
                experiment_cv="{}_{}".format(experiment,fold)

                x_train, y_train, \
                x_valid, y_valid, \
                x_test, y_test = load_fold(hdf5["patients"], exp_storage, fold)
                y_train,y_valid,y_test=y_train.reshape(y_train.shape[0],1),y_valid.reshape(y_valid.shape[0],1),y_test.reshape(y_test.shape[0],1)
                x_train, y_train,x_valid, y_valid,x_test, y_test=list(x_train), list(y_train),list(x_valid), list(y_valid),list(x_test),list(y_test)    
                X=x_train[:]
                Y=y_train[:]
                for i in x_valid:
                    X.append(i)
                for i in y_valid:
                    Y.append(i)
                for i in x_test:
                    X.append(i)
                for i in y_test:
                    Y.append(i)
                X=np.array(X)
                Y=np.array(Y)
                break
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.15,shuffle=True,random_state=1)          

p=n=0
for i in Y_train:
    if i[0]==1:
        p=p+1
    else:
        n=n+1
print('Training Examples: p-{} n-{}'.format(p,n))
p=n=0
for i in Y_test:
    if i[0]==1:
        p=p+1
    else:
        n=n+1
print('Testing Examples:  p-{} n-{}'.format(p,n))

Training Examples: p-460 n-419
Testing Examples:  p-70 n-86


In [3]:
def train_and_evaluate_model(x_train,y_train,x_test,y_test,i):
    #callbacks
    tensorboard=TensorBoard(log_dir='./logs/Exp2/fold_{}'.format(i))
    Results={}
    #Autoencoder-1 architecture
    print("Constructing ae1 architecture...",end="")
    input_size=x_train.shape[1]
    ae1_input_layer=Input((input_size))
    ae1_input_layer_noise=GaussianNoise(0.1)(ae1_input_layer)
    ae1_dropout1=Dropout(0.2)(ae1_input_layer_noise)
    ae1_bottle_neck=Dense(1000,activation='relu')(ae1_dropout1)
    ae1_dropout2=Dropout(0.3)(ae1_bottle_neck)
    ae1_ouput_layer=Dense(input_size,activation='relu')(ae1_bottle_neck)
    ae1=Model(ae1_input_layer,ae1_ouput_layer)
    print("done!")
    print("Configuring ae1...",end="")
    ae1.compile(optimizer='adam',loss='mse',metrics=['mse','mae'])
    print("done!")

    #Traininig ae1
    print("Training ae1...")
    ae1.fit(x_train,x_train,epochs=50,batch_size=16,validation_split=0.1,shuffle=True)
    print("done!")
    print("Evaluating and storing results of ae1...",end="")
    temp=ae1.evaluate(x_test,x_test,verbose=2)
    Results['ae1_loss'],Results['ae1_mse'],Results['ae1_mae']=temp[0],temp[1],temp[2]
    print("done!")
    
    #Saving ae1
    print("Saving ae1...",end="")
    path='SavedModels_In_h5format/Exp2/fold_{}/ae1.hdf5'.format(i)
    ae1.save(path)
    ae1.save('SavedModels_In_SavedModels/Exp2/fold_{}/ae1'.format(i))
    print("done!")
    
    #Preparing data for autoencoder-2
    print("Preparing data for ae2 by loading ae1...",end="")
    ae1_new=tf.keras.models.load_model('SavedModels_In_SavedModels/Exp2/fold_{}/ae1'.format(i))
    ae1_encoder=Model(ae1_input_layer,ae1_bottle_neck)
    ae1_encoder.set_weights([ae1_new.get_weights()[0],ae1_new.get_weights()[1]])
    ae2_X_train=ae1_encoder.predict(x_train)
    ae2_X_test=ae1_encoder.predict(x_test)
    print("done!")

    #Autoencoder-2 architecture
    print("Constructing ae2 architecture...",end="")
    input_size=ae2_X_train.shape[1]
    ae2_input_layer=Input((input_size))
    ae2_input_layer_noise=GaussianNoise(0.2)(ae2_input_layer)
    ae1_dropout1=Dropout(0.2)(ae2_input_layer_noise)
    ae2_bottle_neck=Dense(600,activation='relu')(ae2_input_layer)
    ae1_dropout2=Dropout(0.3)(ae2_bottle_neck)
    ae2_ouput_layer=Dense(input_size,activation='relu')(ae2_bottle_neck)
    ae2=Model(ae2_input_layer,ae2_ouput_layer)
    print("done!")
    print("Configuring ae2...",end="")
    ae2.compile(optimizer='adam',loss='mse',metrics=['mse','mae'])
    print("done!")
    
    #Training ae2
    print("Training ae1...",end="")
    ae2.fit(ae2_X_train,ae2_X_train,epochs=100,batch_size=16,validation_split=0.1,shuffle=True)
    print("done!")
    print("Evaluating and storing results of ae2...",end="")
    temp=ae2.evaluate(ae2_X_test,ae2_X_test,verbose=2)
    Results['ae2_loss'],Results['ae2_mse'],Results['ae2_mae']=temp[0],temp[1],temp[2]
    print("done!")
    #Saving ae2
    print("Saving ae1...",end="")
    ae2.save('SavedModels_In_h5format/Exp2/fold_{}/ae2.hdf5'.format(i))
    ae2.save('SavedModels_In_SavedModels/Exp2/fold_{}/ae2'.format(i))
    print("done!")
    
    #Loading Models for Transfer Learning
    print("Loading ae2")
    ae2_new=tf.keras.models.load_model('SavedModels_In_SavedModels/Exp2/fold_{}/ae2'.format(i))
    print("done!")
    #MLP
    print("Constructing architecture of mlp...",end="")
    mlp=Sequential()
    mlp.add(Dense(1000,activation='relu',input_dim=19900))
    mlp.add(Dense(600,activation='relu'))
    mlp.add(Dense(1,activation='sigmoid'))
    print("done!")
    print("Configuring mlp...",end="")
    mlp.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    print("done!")
    print("Freezing layers of mlp...",end="")
    mlp.layers[0].set_weights([ae1_new.get_weights()[0],ae1_new.get_weights()[1]])
    mlp.layers[0].trainable=False
    mlp.layers[1].set_weights([ae2_new.get_weights()[0],ae2_new.get_weights()[1]])
    mlp.layers[1].trainable=False
    print("done!")
    mlp.summary()
    print("Training mlp...",end="")
    mlp.fit(X_train,Y_train,epochs=100,batch_size=16,validation_split=0.1,shuffle=True, callbacks=[tensorboard])
    print("done!")
    
    print("Saving mlp...",end="")
    ae2.save('SavedModels_In_h5format/Exp2/fold_{}/mlp.hdf5'.format(i))
    ae2.save('SavedModels_In_SavedModels/Exp2/fold_{}/mlp'.format(i))
    print("done!")
    
    print("Evaluating and storing results of mlp...",end="")
    temp=mlp.evaluate(X_test,Y_test,verbose=2)
    Results['mlp_loss'],Results['mlp_accuracy']=temp[0],temp[1]
    print("done!")
    return Results

In [4]:
n_folds=10
i=0
skf = StratifiedKFold(10, shuffle=True)
for i, (train, test) in enumerate(skf.split(X, Y)):
        print("Running Fold", i+1, "/", n_folds)
        x_train=np.array([X[i] for i in train])
        y_train=np.array([Y[i] for i in train])
        x_test=np.array([X[i] for i in test])
        y_test=np.array([Y[i] for i in test])
        print(train_and_evaluate_model(x_train,y_train,x_test,y_test,i+1))
        i=i+1

Running Fold 1 / 10
Constructing ae1 architecture...done!
Configuring ae1...done!
Training ae1...
Train on 837 samples, validate on 94 samples
Epoch 1/50
837/837 [==============================] - 11s 13ms/sample - loss: 0.0459 - mse: 0.0459 - mae: 0.1710 - val_loss: 0.0435 - val_mse: 0.0435 - val_mae: 0.1666
Epoch 2/50
837/837 [==============================] - 10s 12ms/sample - loss: 0.0440 - mse: 0.0440 - mae: 0.1675 - val_loss: 0.0431 - val_mse: 0.0431 - val_mae: 0.1658
Epoch 3/50
837/837 [==============================] - 11s 13ms/sample - loss: 0.0431 - mse: 0.0431 - mae: 0.1658 - val_loss: 0.0425 - val_mse: 0.0425 - val_mae: 0.1648
Epoch 4/50
837/837 [==============================] - 11s 13ms/sample - loss: 0.0425 - mse: 0.0425 - mae: 0.1645 - val_loss: 0.0421 - val_mse: 0.0421 - val_mae: 0.1639
Epoch 5/50
837/837 [==============================] - 10s 12ms/sample - loss: 0.0422 - mse: 0.0422 - mae: 0.1638 - val_loss: 0.0420 - val_mse: 0.0420 - val_mae: 0.1635
Epoch 6/50
837/83

Epoch 49/50
837/837 [==============================] - 11s 13ms/sample - loss: 0.0411 - mse: 0.0411 - mae: 0.1616 - val_loss: 0.0415 - val_mse: 0.0415 - val_mae: 0.1627
Epoch 50/50
837/837 [==============================] - 11s 13ms/sample - loss: 0.0411 - mse: 0.0411 - mae: 0.1616 - val_loss: 0.0417 - val_mse: 0.0417 - val_mae: 0.1631
done!
Evaluating and storing results of ae1...104/1 - 0s - loss: 0.0398 - mse: 0.0419 - mae: 0.1636
done!
Saving ae1...

W1219 20:14:43.305998 140561518417664 deprecation.py:506] From /home/ashok/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


done!
Preparing data for ae2 by loading ae1...done!
Constructing ae2 architecture...done!
Configuring ae2...done!
Training ae1...Train on 837 samples, validate on 94 samples
Epoch 1/100
837/837 [==============================] - 1s 894us/sample - loss: 0.0811 - mse: 0.0811 - mae: 0.0375 - val_loss: 0.0275 - val_mse: 0.0275 - val_mae: 0.0197
Epoch 2/100
837/837 [==============================] - 0s 418us/sample - loss: 0.0280 - mse: 0.0280 - mae: 0.0164 - val_loss: 0.0108 - val_mse: 0.0108 - val_mae: 0.0076
Epoch 3/100
837/837 [==============================] - 0s 446us/sample - loss: 0.0119 - mse: 0.0119 - mae: 0.0061 - val_loss: 0.0061 - val_mse: 0.0061 - val_mae: 0.0031
Epoch 4/100
837/837 [==============================] - 0s 449us/sample - loss: 0.0097 - mse: 0.0097 - mae: 0.0038 - val_loss: 0.0059 - val_mse: 0.0059 - val_mae: 0.0027
Epoch 5/100
837/837 [==============================] - 0s 448us/sample - loss: 0.0096 - mse: 0.0096 - mae: 0.0035 - val_loss: 0.0059 - val_mse: 0.0059

837/837 [==============================] - 0s 439us/sample - loss: 0.0096 - mse: 0.0096 - mae: 0.0034 - val_loss: 0.0059 - val_mse: 0.0059 - val_mae: 0.0026
Epoch 49/100
837/837 [==============================] - 0s 442us/sample - loss: 0.0096 - mse: 0.0096 - mae: 0.0036 - val_loss: 0.0059 - val_mse: 0.0059 - val_mae: 0.0026
Epoch 50/100
837/837 [==============================] - 0s 438us/sample - loss: 0.0096 - mse: 0.0096 - mae: 0.0034 - val_loss: 0.0059 - val_mse: 0.0059 - val_mae: 0.0027
Epoch 51/100
837/837 [==============================] - 0s 441us/sample - loss: 0.0096 - mse: 0.0096 - mae: 0.0035 - val_loss: 0.0059 - val_mse: 0.0059 - val_mae: 0.0027
Epoch 52/100
837/837 [==============================] - 0s 446us/sample - loss: 0.0096 - mse: 0.0096 - mae: 0.0035 - val_loss: 0.0060 - val_mse: 0.0060 - val_mae: 0.0028
Epoch 53/100
837/837 [==============================] - 0s 437us/sample - loss: 0.0096 - mse: 0.0096 - mae: 0.0035 - val_loss: 0.0059 - val_mse: 0.0059 - val_mae: 

...

Epoch 29/50
838/838 [==============================] - 11s 14ms/sample - loss: 0.0412 - mse: 0.0412 - mae: 0.1617 - val_loss: 0.0414 - val_mse: 0.0414 - val_mae: 0.1627
Epoch 30/50
838/838 [==============================] - 11s 13ms/sample - loss: 0.0411 - mse: 0.0411 - mae: 0.1616 - val_loss: 0.0414 - val_mse: 0.0414 - val_mae: 0.1626
Epoch 31/50
838/838 [==============================] - 12s 15ms/sample - loss: 0.0412 - mse: 0.0412 - mae: 0.1617 - val_loss: 0.0413 - val_mse: 0.0413 - val_mae: 0.1622
Epoch 32/50
838/838 [==============================] - 12s 14ms/sample - loss: 0.0411 - mse: 0.0411 - mae: 0.1616 - val_loss: 0.0413 - val_mse: 0.0413 - val_mae: 0.1622
Epoch 33/50
838/838 [==============================] - 12s 14ms/sample - loss: 0.0411 - mse: 0.0411 - mae: 0.1616 - val_loss: 0.0413 - val_mse: 0.0413 - val_mae: 0.1624
Epoch 34/50
838/838 [==============================] - 11s 14ms/sample - loss: 0.0411 - mse: 0.0411 - mae: 0.1616 - val_loss: 0.0413 - val_mse: 0.0413 - va

Epoch 26/100
838/838 [==============================] - 0s 536us/sample - loss: 0.0104 - mse: 0.0104 - mae: 0.0028 - val_loss: 0.0045 - val_mse: 0.0045 - val_mae: 0.0021
Epoch 27/100
838/838 [==============================] - 0s 513us/sample - loss: 0.0104 - mse: 0.0104 - mae: 0.0029 - val_loss: 0.0045 - val_mse: 0.0045 - val_mae: 0.0021
Epoch 28/100
838/838 [==============================] - 0s 584us/sample - loss: 0.0104 - mse: 0.0104 - mae: 0.0029 - val_loss: 0.0045 - val_mse: 0.0045 - val_mae: 0.0021
Epoch 29/100
838/838 [==============================] - 0s 469us/sample - loss: 0.0104 - mse: 0.0104 - mae: 0.0028 - val_loss: 0.0045 - val_mse: 0.0045 - val_mae: 0.0021
Epoch 30/100
838/838 [==============================] - 0s 474us/sample - loss: 0.0104 - mse: 0.0104 - mae: 0.0028 - val_loss: 0.0045 - val_mse: 0.0045 - val_mae: 0.0021
Epoch 31/100
838/838 [==============================] - 0s 529us/sample - loss: 0.0104 - mse: 0.0104 - mae: 0.0029 - val_loss: 0.0045 - val_mse: 0.004

838/838 [==============================] - 0s 479us/sample - loss: 0.0105 - mse: 0.0105 - mae: 0.0032 - val_loss: 0.0046 - val_mse: 0.0046 - val_mae: 0.0024
Epoch 75/100
838/838 [==============================] - 0s 452us/sample - loss: 0.0105 - mse: 0.0105 - mae: 0.0031 - val_loss: 0.0046 - val_mse: 0.0046 - val_mae: 0.0024
Epoch 76/100
838/838 [==============================] - 0s 463us/sample - loss: 0.0105 - mse: 0.0105 - mae: 0.0030 - val_loss: 0.0046 - val_mse: 0.0046 - val_mae: 0.0023
Epoch 77/100
838/838 [==============================] - 0s 457us/sample - loss: 0.0105 - mse: 0.0105 - mae: 0.0030 - val_loss: 0.0045 - val_mse: 0.0045 - val_mae: 0.0022
Epoch 78/100
838/838 [==============================] - 0s 452us/sample - loss: 0.0104 - mse: 0.0104 - mae: 0.0029 - val_loss: 0.0045 - val_mse: 0.0045 - val_mae: 0.0022
Epoch 79/100
838/838 [==============================] - 0s 456us/sample - loss: 0.0104 - mse: 0.0104 - mae: 0.0029 - val_loss: 0.0045 - val_mse: 0.0045 - val_mae: 

791/791 [==============================] - 4s 5ms/sample - loss: 7.1671e-04 - accuracy: 1.0000 - val_loss: 1.5325 - val_accuracy: 0.6477
Epoch 19/100
791/791 [==============================] - 4s 5ms/sample - loss: 6.7831e-04 - accuracy: 1.0000 - val_loss: 1.7728 - val_accuracy: 0.6250
Epoch 20/100
791/791 [==============================] - 4s 5ms/sample - loss: 2.0890e-04 - accuracy: 1.0000 - val_loss: 1.6358 - val_accuracy: 0.6364
Epoch 21/100
791/791 [==============================] - 4s 5ms/sample - loss: 1.0152e-04 - accuracy: 1.0000 - val_loss: 1.6491 - val_accuracy: 0.6364
Epoch 22/100
791/791 [==============================] - 5s 6ms/sample - loss: 8.6973e-05 - accuracy: 1.0000 - val_loss: 1.6576 - val_accuracy: 0.6364
Epoch 23/100
791/791 [==============================] - 4s 6ms/sample - loss: 7.7339e-05 - accuracy: 1.0000 - val_loss: 1.6640 - val_accuracy: 0.6364
Epoch 24/100
791/791 [==============================] - 4s 5ms/sample - loss: 6.9676e-05 - accuracy: 1.0000 - val

Epoch 73/100
791/791 [==============================] - 4s 5ms/sample - loss: 5.0083e-06 - accuracy: 1.0000 - val_loss: 1.9709 - val_accuracy: 0.6477
Epoch 74/100
791/791 [==============================] - 4s 5ms/sample - loss: 4.8128e-06 - accuracy: 1.0000 - val_loss: 1.9760 - val_accuracy: 0.6364
Epoch 75/100
791/791 [==============================] - 4s 5ms/sample - loss: 4.6443e-06 - accuracy: 1.0000 - val_loss: 1.9815 - val_accuracy: 0.6364
Epoch 76/100
791/791 [==============================] - 4s 5ms/sample - loss: 4.4477e-06 - accuracy: 1.0000 - val_loss: 1.9864 - val_accuracy: 0.6477
Epoch 77/100
791/791 [==============================] - 4s 5ms/sample - loss: 4.2771e-06 - accuracy: 1.0000 - val_loss: 1.9922 - val_accuracy: 0.6364
Epoch 78/100
791/791 [==============================] - 4s 5ms/sample - loss: 4.1180e-06 - accuracy: 1.0000 - val_loss: 1.9972 - val_accuracy: 0.6364
Epoch 79/100
791/791 [==============================] - 4s 5ms/sample - loss: 3.9616e-06 - accuracy:

In [ ]:
#fold_1
{'ae1_loss': 0.041857508226082876, 'ae1_mse': 0.041857507, 'ae1_mae': 0.16361333, 'ae2_loss': 0.0059846284297796395, 
 'ae2_mse': 0.0059846286, 'ae2_mae': 0.0029167475, 
 'mlp_loss': 2.682267745335897, 'mlp_accuracy': 0.6666667}
#fold_2
{'ae1_loss': 0.04151521279261662, 'ae1_mse': 0.041515213, 'ae1_mae': 0.16270886, 'ae2_loss': 0.0020301583437965466,
 'ae2_mse': 0.0020301584, 'ae2_mae': 0.0019260847, 
 'mlp_loss': 2.507642159095177, 'mlp_accuracy': 0.6666667}

#fold_3
{'ae1_loss': 0.04264852891747768, 'ae1_mse': 0.04264853, 'ae1_mae': 0.16460998, 'ae2_loss': 0.0008832786250143097, 
 'ae2_mse': 0.00088327867, 'ae2_mae': 0.0015826778, 
 'mlp_loss': 2.551986168592404, 'mlp_accuracy': 0.6666667}

#fold_4
{'ae1_loss': 0.041090411062424, 'ae1_mse': 0.04109041, 'ae1_mae': 0.16196603, 'ae2_loss': 0.002791929327381345,
 'ae2_mse': 0.002791929, 'ae2_mae': 0.0020756198, 
 'mlp_loss': 2.9056687966371193, 'mlp_accuracy': 0.6730769}

#fold_5
{'ae1_loss': 0.04139249943769895, 'ae1_mse': 0.0413925, 'ae1_mae': 0.16164924, 'ae2_loss': 0.006959259151839293,
 'ae2_mse': 0.0069592595, 'ae2_mae': 0.00350656, 
 'mlp_loss': 2.602505628879254, 'mlp_accuracy': 0.6858974}

#fold_6
{'ae1_loss': 0.042703317568718806, 'ae1_mse': 0.042703312, 'ae1_mae': 0.16532052, 'ae2_loss': 0.007893176884978141, 
 'ae2_mse': 0.007893178, 'ae2_mae': 0.0034152337, 
 'mlp_loss': 2.581120546047504, 'mlp_accuracy': 0.6474359}

#fold_7
{'ae1_loss': 0.04116378822228284, 'ae1_mse': 0.041163787, 'ae1_mae': 0.1622161, 'ae2_loss': 0.007032900463192932, 
 'ae2_mse': 0.0070329006, 'ae2_mae': 0.0027305223, 
 'mlp_loss': 3.061433896040305, 'mlp_accuracy': 0.6858974}

#fold_8
{'ae1_loss': 0.04226284173941149, 'ae1_mse': 0.04226284, 'ae1_mae': 0.16429321, 'ae2_loss': 0.0006639336716740853,
 'ae2_mse': 0.0006639337, 'ae2_mae': 0.0011286066, 
 'mlp_loss': 2.2080062413826966, 'mlp_accuracy': 0.6858974}

#fold_9
{'ae1_loss': 0.043227021519130875, 'ae1_mse': 0.043227024, 'ae1_mae': 0.16639249, 'ae2_loss': 0.0066035505546962175, 
 'ae2_mse': 0.00660355, 'ae2_mae': 0.002906847, 
 'mlp_loss': 2.641597277078873, 'mlp_accuracy': 0.6923077}

#fold_10
{'ae1_loss': 0.042155014296758525, 'ae1_mse': 0.042155012, 'ae1_mae': 0.16382372, 'ae2_loss': 0.004541662444803611, 
 'ae2_mse': 0.0045416625, 'ae2_mae': 0.0024602918,
 'mlp_loss': 2.450022801374778, 'mlp_accuracy': 0.6730769}


In [5]:
import numpy as np
avg_acc=np.array([0.6666667,0.6666667,0.6666667,0.6730769,0.6858974,0.6474359,0.6858974,0.6858974,0.6923077,0.6730769])
avg_acc=(np.sum(avg_acc)/10)*100
print(avg_acc)

67.43589699999998


In [ ]:
input_size=x_train.shape[1]
ae1_input_layer=Input((input_size))
ae1_input_layer_noise=GaussianNoise(0.1)(ae1_input_layer)
ae1_dropout1=Dropout(0.2)(ae1_input_layer_noise)
ae1_bottle_neck=Dense(1000,activation='relu')(ae1_dropout1)
ae1_dropout2=Dropout(0.3)(ae1_bottle_neck)
ae1_ouput_layer=Dense(input_size,activation='relu')(ae1_bottle_neck)
ae1=Model(ae1_input_layer,ae1_ouput_layer)
ae1_encoder=Model(ae1_input_layer,ae1_bottle_neck)
ae1_encoder.summary()
ae1_encoder.get_weights()[1].shape